# Using GeoJSON Point Features with Markers

In [ ]:
import os
import folium
import geopandas as gpd

In [ ]:
rootpath = os.path.abspath(os.getcwd())

In [ ]:
gdf = gpd.read_file(os.path.join(rootpath, "data", "subwaystations.geojson"))

In [ ]:
gdf.head()

In [ ]:
gdf['href'] = '<a href="' + gdf.url + '">' + gdf.url + "</a>"
gdf['service_level'] = gdf.notes.str.split(', ').apply(lambda x: len([v for v in x if "all" in v]))
gdf['lines_served'] = gdf.line.str.split('-').apply(lambda x: len(x))

In [ ]:
service_levels = gdf.service_level.unique().tolist()
service_levels

In [ ]:
colors = ["orange", "yellow", "green", "blue"]

# Use a Circle as a Marker

In [ ]:
m = folium.Map(
    location=[40.75, -73.95],
    zoom_start=13)

folium.GeoJson(
    gdf,
    name="Subway Stations",
    marker=folium.Circle(radius=4, fill_color="orange", fill_opacity=0.4, color="black", weight=1),
    tooltip=folium.GeoJsonTooltip(fields=["name", "line", "notes"]),
    zoom_on_click=True,
    popup=folium.GeoJsonPopup(fields=["name", "line", "href", "notes"]),
    style_function=lambda x: {
        "markerColor": "yellow",
        "fillColor": colors[x['properties']['service_level']],
        "radius": (x['properties']['lines_served'])*30},
    highlight_function=lambda x: {"fillOpacity": 0.8}).add_to(m)

m

## Or use a DivIcon

In [ ]:
m = folium.Map(location=[40.75, -73.95], zoom_start=13)


def style_function(feature):
    props = feature.get('properties')
    markup = f"""
        <a href="{props.get('url')}">
            <div style="font-size: 0.8em;">
            <div style="width: 10px;
                        height: 10px;
                        border: 1px solid black;
                        border-radius: 5px;
                        background-color: orange;">
            </div>
            {props.get('name')}
        </div>
        </a>
    """
    return {"html": markup}


folium.GeoJson(
    gdf,
    name="Subway Stations",
    marker=folium.Marker(icon=folium.DivIcon()),
    tooltip=folium.GeoJsonTooltip(fields=["name", "line", "notes"]),
    zoom_on_click=True,
    popup=folium.GeoJsonPopup(fields=["name", "line", "href", "notes"]),
    style_function=style_function).add_to(m)

m